#Setup our parameters & keys

In [ ]:
import pandas as pd
import requests
import json
import numpy as np


api_id='api_id'
api_secret_key='api_secret'
org_key='org_key'
base_url='https://defense.conferdeploy.net'

#Alternatively you can prompt each time for information:
#api_id = input('Enter the CBC API ID: ')
#api_secret_key = input('Enter the CBC API Key: ')
#org_key = input('Enter the CBC Org Key: ')
#base_url = input('Enter the URL of your Carbon Black Cloud instance: ')

api_token= '/'.join([api_secret_key, api_id])

print('Your Parameters')
print('API ID: ' + api_id)
print('Secret: ' + api_secret_key)
print('Org key: ' + org_key)
print('Base URL: ' + base_url)
print('API token: ' + api_token)

#Get list of endpoints:

In [ ]:
#Form our request URL:
req_url = base_url + '/appservices/v6/orgs/' + org_key + '/devices/_search'

#Set our headers
headers = {'X-Auth-Token': api_token,
           'Content-Type': 'application/json'
           }
#Set our data (body) for the request
data = '{"criteria":{"deployment_type":["ENDPOINT","WORKLOAD","VDI"]},"sort":[{"field":"last_contact_time","order":"DESC"}],"start":1,"rows":10000,"":""}'

#Double check they're ok
print('Request URL: ' + req_url)
print('Headers: ', end="") 
print(headers)
print('Data: ', end="")
print(data)

In [ ]:
#Make the request
req = requests.post(req_url, headers=headers, data=data)
print('Status code: ' + str(req.status_code))


In [ ]:
devices_dict = req.json()
devices_dict['results']
devices = pd.DataFrame.from_dict(devices_dict['results'])
devices.set_index('device_owner_id', drop=True, inplace=True)

print('Total devices found: ', end="")
print(devices_dict.get('num_found'))
devices.head()

In [ ]:
#Cool. Let's export to CSV now
devices.to_csv('devices.csv')

#Get Alerts


In [ ]:
#Form our request URL:
req_url = base_url + '/appservices/v6/orgs/' + org_key + '/alerts/_search'

#Set our headers
headers = {'X-Auth-Token': api_token,
           'Content-Type': 'application/json'
           }
#Set our data (body) for the request
data = '{"criteria":{"group_results":"false","minimum_severity":"1","category":["THREAT"],"workflow":["DISMISSED"],"alert_type":["CB_ANALYTICS"],"create_time":{"range":"all"}},"query":"","sort":[{"field":"create_time","order":"DESC"}],"start":0,"rows":10000}'

#Double check they're ok
print('Request URL: ' + req_url)
print('Headers: ', end="") 
print(headers)
print('Data: ', end="")
print(data)

In [ ]:
#Make the request
req = requests.post(req_url, headers=headers, data=data)
print('Status code: ' + str(req.status_code))


In [ ]:
#Let's see what we've got
alerts_dict = req.json()

print('Total alerts found: ', end="")
print(alerts_dict.get('num_found'))
print('Total alerts available: ', end="")
print(alerts_dict.get('num_available'))

alerts_dict['results']
alerts = pd.DataFrame.from_dict(alerts_dict['results'])
alerts.head()

In [ ]:
#Cool. Let's export to CSV now
alerts.to_csv('alerts.csv')

In [ ]:
sev_summary = pd.DataFrame(alerts.pivot_table(index='severity',aggfunc='count').id).rename(columns = {'id':'Total_Alerts'})
sev_summary

In [ ]:
sev_summary.plot(kind='bar')

#Get Users

In [ ]:
#Form our request URL:
req_url = base_url + '/appservices/v6/orgs/' + org_key + '/users'

#Set our headers
headers = {'X-Auth-Token': api_token,
           'Content-Type': 'application/json'
           }

#Double check they're ok
print('Request URL: ' + req_url)
print('Headers: ', end="") 
print(headers)

In [ ]:
#Make the request
req = requests.get(req_url, headers=headers)
print('Status code: ' + str(req.status_code))


In [ ]:
#Let's see what we've got
users_dict = req.json()
users_dict['users']
users = pd.DataFrame.from_dict(users_dict['users'])
users

In [ ]:
#Due to the "DEPRECATED" values above, we need to look up each user ("principal") against a different API endpoint and see their "grant" (role)

# See here for more info: https://developer.carbonblack.com/reference/carbon-black-cloud/platform/latest/access-profiles-and-grants/#get-grant-of-a-principal

#For each login_id in users, look up the principal's grant:
for i, j in users.iterrows():
  print('login_id ' + str(j.login_id) + ' ', end="")
  
  #Form our request URL:
  req_url = base_url + '/access/v2/orgs/' + org_key + '/grants/psc:user:' + org_key + ':' + str(j.login_id)

  #Set our headers
  headers = {'X-Auth-Token': api_token,
           'Content-Type': 'application/json'
           }

  #Make the request
  req = requests.get(req_url, headers=headers)
  print('Status code: ' + str(req.status_code) + ' ', end="")

  #Let's see what we've got
  principal_dict = req.json()
  role=np.array(principal_dict['roles'])

  #Clean up the results
  role=str(role).replace("BETA_","")
  role=role.replace("psc:role::","")
  role=role.replace("[","")
  role=role.replace("]","")
  role=role.replace("'","")
  role=role.replace("_", " ")

  print(role)
  
  #Set the new role value in the users dataframe
  users.at[i,'role']=role
  print()

  
  

In [ ]:
#Let's see the real user roles, now updated in the dataframe
users

In [223]:
#Cool. Let's export to CSV now
users.to_csv('users.csv')

#Get USB Devices


In [242]:
#Looks like a 2-step process, similar to looking up users.
# POST {cbc-hostname}/device_control/v3/orgs/{org_key}/devices/_search

#Form our request URL:
req_url = base_url + '/device_control/v3/orgs/' + org_key + '/devices/_search'

#Set our headers
headers = {'X-Auth-Token': api_token,
           'Content-Type': 'application/json'
           }
#Set our data (body) for the request
data = '{"query":"","criteria":{},"sort":[{"field":"last_seen","order":"DESC"}],"start":0,"rows":10000}'

#Double check they're ok
print('Request URL: ' + req_url)
print('Headers: ', end="") 
print(headers)
print('Data: ', end="")
print(data)

Request URL: https://defense.conferdeploy.net/device_control/v3/orgs/7PESY63N/devices/_search
Headers: {'X-Auth-Token': '5PQFBDZEFFG8MGTZEGHC7Z5Q/3GVS5NZJCK', 'Content-Type': 'application/json'}
Data: {"query":"","criteria":{},"sort":[{"field":"last_seen","order":"DESC"}],"start":0,"rows":10000}


In [243]:
#Make the request
req = requests.post(req_url, headers=headers, data=data)
print('Status code: ' + str(req.status_code))

Status code: 200


In [244]:
usb_devices_dict = req.json()
usb_devices_dict['results']
usb_devices = pd.DataFrame.from_dict(usb_devices_dict['results'])
usb_devices.set_index('id', drop=True, inplace=True)
usb_devices

,first_seen,last_seen,vendor_name,vendor_id,product_name,product_id,serial_number,last_endpoint_name,last_endpoint_id,last_policy_id,endpoint_count,device_friendly_name,device_name,created_at,updated_at,status
id,,,,,,,,,,,,,,,,
f85444e7-62ea-3a02-88fa-2047f7512b72,2021-10-30T17:08:32Z,2021-10-30T17:08:32Z,Kingston Technology Company Inc.,0x13FE,USB DISK 3.0,0x6300,P190306607009A7986E5D713,windows81,3220244,72728,1,UFD 3.0 Silicon-Power32G USB Device,\Device\HarddiskVolume4,2021-10-30T17:10:43Z,2021-10-30T17:10:43Z,UNAPPROVED
701a8dd4-cfb9-3435-a372-216f64510d2c,2021-10-29T20:09:26Z,2021-10-29T20:09:26Z,SanDisk,0x0781,Ultra Fit,0x5583,05018891781b17ea35b158d3e437234b1ae2d89bfbb754...,windows81,3220244,72728,1,SanDisk Ultra Fit USB Device,\Device\HarddiskVolume2,2021-10-29T20:09:38Z,2021-10-29T20:09:38Z,UNAPPROVED
2e2d18de-d624-396f-942e-c8b82e15645c,2021-03-18T17:26:14Z,2021-04-14T12:45:22Z,Kingston Technology Company Inc.,0x13FE,USB DISK 3.0,0x6300,070002476C3DA691,DESKTOP-7HPGFA3,4123530,65848,1,USB DISK 3.0 USB Device,\Device\HarddiskVolume4,2021-03-18T17:26:19Z,2021-04-14T12:51:26Z,UNAPPROVED
